In [1]:

import jax.numpy as jnp

import sys
sys.path.insert(1,'../scripts/')

import utils
import loader
import tqdm
%load_ext autoreload
%autoreload 2
import imp
imp.reload(loader)
import setup
import scipy.stats as sts
from matplotlib import pyplot as plt

import numpy as np

In [ ]:

params = {
    'file': '../../../Data/',
    'tag': '2022_Q2_IBL_et_al_RepeatedSite',
    'probe': 'probe00',
    # 'sessions': [5,6,7,8,9,10,11,12],
    'areas': ['CA1','DG','LP','PO','VISa'],
    'props':{'train':.5,'test':.5,'validation':0},
    'seeds':{'train':0,'test':1,'validation':2},
    'n_neurons': None, # all neurons
    'n_trials': None, # all trials
    'pre_time':1,
    'post_time':1,
    'n_bins': 150,
    'align_to': 'stim',
    'train_trial_prop':.9, 
    'train_condition_prop':1, 
    'seed':0,
    'verbose': False,
    'bins_as_conds': False
}

dataloader = loader.IBLDataLoader(
    params,eids=setup.good_eids,parallel=False
)



In [ ]:
xs,ys,rs,cs = dataloader.load_train_data()
ys = [np.array(ys[n]) for n in range(len(ys))]

In [ ]:
print(len(ys))  # Number of subjects?
print(len(ys[0]))  # Number of time bins for this subject
print(len(ys[0][0]))  # Number of trials for neuron 0 of subject 0?
print(len(ys[0][0][0]))  # train_trial_prop? Number of conditions?
print(len(ys[0][0][0][0]) ) # Number of neurons ?

In [ ]:
print(len(rs))  # Number of subjects?
print(len(rs[0]))  # Number of time bins for this subject
print(len(rs[0][0]))  # Number of trials for neuron 0 of subject 0?
print(len(rs[0][0][0]))  # train_trial_prop? Number of conditions?

In [ ]:
print(len(ys))  # Number of subjects?
print(len(ys[1]))  # Number of time bins for this subject/trials?
print(len(ys[1][0]))  # Number of trials for neuron 0 of subject 0?
print(len(ys[1][0][0]))  # Number of conditions?
print(len(ys[1][0][0][0]) ) # Number of neurons ?

In [ ]:
# save data 
# with(open("25_avg_folds_stim_aligned.pkl","wb")) as f:
#     import pickle
#     pickle.dump([params,_ys,train_data_folds,test_data_folds],f)
    

In [2]:
params = {
    'file': '../../../Data/',
    'tag': '2022_Q2_IBL_et_al_RepeatedSite',
    'probe': 'probe00',
    'sessions': [0,5,6],
    'areas': ['CA1','DG','LP','PO','VISa'],
    'props':{'train':.5,'test':.5,'validation':0},
    'seeds':{'train':0,'test':1,'validation':2},
    'n_neurons': None, # all neurons
    'n_trials': None, # all trials
    'pre_time':0,
    'post_time':0.4,
    'n_bins': 10,
    'align_to': 'response',
    'train_trial_prop':.9, 
    'train_condition_prop':1, 
    'seed':0,
    'verbose': False,
    'bins_as_conds': True
    }

dataloader = loader.IBLDataLoader(
         params,eids=setup.good_eids
)


/Users/laurinemarty/miniconda3/envs/project/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/Users/laurinemarty/miniconda3/envs/project/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/Users/laurinemarty/miniconda3/envs/project/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2023-04-05"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/Users/laurinemarty/miniconda3/envs/project/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/Users/laurinemarty/miniconda3/envs/project/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multipl

In [ ]:
# I dont know why n_folds = 1
n_folds = 100

train_data_folds, test_data_folds = dataloader.new_folds(n_folds)

__train_data_folds = []
__test_data_folds = []

for i in range(n_folds):
    _,ys,_,cs = train_data_folds[i]
    ys = [np.array(ys[n]) for n in range(len(ys))]

    _,ys_t,_,cs_t = test_data_folds[i]
    ys_t = [np.array(ys_t[n]) for n in range(len(ys_t))]
    
    # filter bad neurons/sessions
    min_neurons = 100
    min_trials = 20

    valid = [i for i in range(len(ys)) if ys[i].shape[-1] >= min_neurons and ys[i].shape[0] >= min_trials]
    # sort neurons by lowest std across conditions and select top 100
    ys = [y[:,:,jnp.argsort(y.mean(0).std(0))[:100]] for y in ys]

    # select subjects with enough neurons and trials

    # Averaging across trials -- you get (n_conditions, 100 neurons) per subject
    ys = [ys[i].mean(0) for i in valid]
    cs = [cs[i].mean(0) for i in valid]

    ys_t = [ys_t[i].mean(0) for i in valid]
    cs_t = [cs_t[i].mean(0) for i in valid]

    S = len(ys)
    __train_data_folds.append([ys,cs])
    __test_data_folds.append([ys_t,cs_t])

# # save pickle
import pickle
with open("100_folds_1_time_point.pkl","wb") as f:
    pickle.dump([params,__train_data_folds,__test_data_folds],f)

In [4]:
# I dont know why n_folds = 1
n_folds = 100

train_data_folds, test_data_folds = dataloader.new_folds(n_folds)

__train_data_folds = []
__test_data_folds = []

for i in range(n_folds):
    _,ys,_,cs = train_data_folds[i]
    ys = [np.array(ys[n]) for n in range(len(ys))]

    """print("Neural Data:")
    print(len(ys))  # Number of subjects?
    print(len(ys[0]))  # Number of trials?
    print(len(ys[0][0]))  # Number of time points?
    print(len(ys[0][0][0]))  # Number of neurons?
    print()

    print ("Conditions:")
    print(len(cs))  # Number of subjects?
    print(len(cs[0]))  # Number of trials?
    print(len(cs[0][0]))  # Number of time points?
    print() """

    _,ys_t,_,cs_t = test_data_folds[i]
    ys_t = [np.array(ys_t[n]) for n in range(len(ys_t))]
    """print("Test Neural Data:")
    print(len(ys_t))  # Number of subjects?
    print(len(ys_t[0]))  # Number of trials?
    print(len(ys_t[0][0]))  # Number of time points?
    print(len(ys_t[0][0][0]))  # Number of neurons?
    print()"""
    
    # filter bad neurons/sessions
    min_neurons = 100
    min_trials = 20

    valid = [i for i in range(len(ys)) if ys[i].shape[-1] >= min_neurons and ys[i].shape[0] >= min_trials]
    # sort neurons by lowest std across conditions and select top 100
    ys = [y[:,:,jnp.argsort(y.mean(0).std(0))[:100]] for y in ys]

    """print("Subject 0: sort neurons by lowest std:")
    print(ys[0][:,:,jnp.argsort(ys[0].mean(0).std(0))[:100]].shape)
    print()"""

    # select subjects with enough neurons and trials

    # Averaging across trials -- you get (n_conditions, 100 neurons) per subject
    ys = [ys[i].mean(0) for i in valid]
    cs = [cs[i].mean(0) for i in valid]

    """print("Neural Data after processing:")
    print(len(ys))  # Number of subjects?
    print(len(ys[0]))  # Number of conditions?
    print(len(ys[0][0]))  # Number of neurons?
    print()"""

    ys_t = [ys_t[i].mean(0) for i in valid]
    cs_t = [cs_t[i].mean(0) for i in valid]

    S = len(ys)
    __train_data_folds.append([ys,cs])
    __test_data_folds.append([ys_t,cs_t])

# # save pickle
import pickle
with open("100_folds_1_time_point.pkl","wb") as f:
    pickle.dump([params,__train_data_folds,__test_data_folds],f)

In [ ]:
print(len(ys))  # Number of subjects?
print(len(ys[0]))  # Number of trials?
print(len(ys[0][0]))  # Number of time points?

In [ ]:
print(len(ys))  # Number of subjects?
print(len(ys[1]))  # Number of trials?
print(len(ys[1][0]))  # Number of time points?
#print(len(ys[1][0][0]))  # Number of neurons?
#print(len(ys[1][0][0][0]) ) # Number of neurons ?

In [ ]:
xs,ys,rs,cs = dataloader.load_train_data()

valid_cs = np.array([cs[i] for i in valid])
import pickle
with open("raw_behavior.pkl","wb") as f:
    pickle.dump(valid_cs,f)

In [5]:
xs,ys,rs,cs = dataloader.load_train_data()

#valid_cs = np.array([cs[i] for i in valid])
valid_cs = [np.array(cs[n]) for n in range(len(valid))]

import pickle
with open("raw_behavior.pkl","wb") as f:
    pickle.dump(valid_cs,f)